In [1]:
from pathlib import Path
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, to_json, col, udf, explode, lit, coalesce
import logging
from setLogger import setLogger
from resolvePath import resolvePath
from transform import brands, datasetTransform

In [2]:
spark = (
                SparkSession
                .builder
                .appName("cg-pyspark-assignment-nb")
                .master("local")
                .config("spark.sql.repl.eagerEval.enabled", True)
                .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 18:28:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Create and configure logger
logger = logging.getLogger(__name__)

# Call the setLogger method to instantiate handler
logger_obj = setLogger(logger, 'main')
logger = logger_obj.set_handler()
logger.info('Logger succesfully created')

In [4]:
# Resolve paths
path_obj = resolvePath()
data_dir, clp_path, cogo_path, dats_path, okay_path, spar_path, log_path, postal_config_path = path_obj.get_path()
# Create a transform object and load CLP data into a dataframe
clp_obj = brands(clp_path, spark)
clp_df = clp_obj.get_data_by_brand('CLP')
cogo_obj = brands(cogo_path, spark)
cogo_df = cogo_obj.get_data_by_brand('COGO')
dats_obj = brands(dats_path, spark)
dats_df = dats_obj.get_data_by_brand('DATS')
logger.info('DATS DATS DATS')
logger.info(dats_df.printSchema())
okay_obj = brands(okay_path, spark)
okay_df = okay_obj.get_data_by_brand('OKAY')
spar_obj = brands(spar_path, spark)
spar_df = spar_obj.get_data_by_brand('SPAR')

root
 |-- address: struct (nullable = true)
 |    |-- cityName: string (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- countryName: string (nullable = true)
 |    |-- houseNumber: string (nullable = true)
 |    |-- postalcode: string (nullable = true)
 |    |-- streetName: string (nullable = true)
 |-- branchId: string (nullable = true)
 |-- commercialName: string (nullable = true)
 |-- ensign: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- geoCoordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- handoverServices: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isActive: boolean (nullable = true)
 |-- moreInfoUrl: string (nullable = true)
 |-- placeId: long (nullable = true)
 |-- placeSearchOpeningHours: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c

In [5]:
# temperoryClosures attribute transformation
clp_df2= clp_obj.transform_tempClosure_attr(clp_df)
logger.info(f'CLP DF2 count > {clp_df2.count()}')

cogo_df2= cogo_obj.transform_tempClosure_attr(cogo_df)
logger.info(f'COGO DF2 count > {cogo_df2.count()}')

okay_df2= okay_obj.transform_tempClosure_attr(okay_df)
logger.info(f'OKAY DF2 count > {okay_df2.count()}')

spar_df2= spar_obj.transform_tempClosure_attr(spar_df)
logger.info(f'SPAR DF2 count > {spar_df2.count()}')

dats_df2= dats_obj.transform_tempClosure_attr(dats_df)
logger.info(f'SPAR DF2 count > {dats_df2.count()}')

+---------------------------------------------------+--------+-------------------+-----------------+----------------------+--------------------------------+--------+---------------------------------------------------+-------+---------------------------------------------------------------------------+-------------------+-------------------------------------------------+-------------------+------------+------------------------+-----+----------------------+
|address                                            |branchId|commercialName     |ensign           |geoCoordinates        |handoverServices                |isActive|moreInfoUrl                                        |placeId|placeSearchOpeningHours                                                    |placeType          |routeUrl                                         |sellingPartners    |sourceStatus|temporaryClosures       |brand|temporaryClosures_from|
+---------------------------------------------------+--------+-------------------+

In [6]:
# handoverServices attribute transformation
clp_df3 = clp_obj.explode_array_attribute(clp_df2, 'handoverServices')
cogo_df3 = cogo_obj.explode_array_attribute(cogo_df2, 'handoverServices')
okay_df3 = okay_obj.explode_array_attribute(okay_df2, 'handoverServices')


# sellingPartners attribute transformation
clp_df4 = clp_obj.explode_array_attribute(clp_df3, 'sellingPartners')
cogo_df4 = cogo_obj.explode_array_attribute(cogo_df3, 'sellingPartners')
okay_df4 = okay_obj.explode_array_attribute(okay_df3, 'sellingPartners')

# SPAR add missing columns
spar_add_col_list = ['handoverServices', 'sellingPartners']
spar_df3 = spar_obj.add_default_string_column(spar_df2, spar_add_col_list)

# placeSearchOpeningHours attribute array explode
clp_df5  = clp_obj.explode_array_attribute(clp_df4, 'placeSearchOpeningHours')
cogo_df5 = cogo_obj.explode_array_attribute(cogo_df4, 'placeSearchOpeningHours')
okay_df5 = okay_obj.explode_array_attribute(okay_df4, 'placeSearchOpeningHours')
spar_df4 = spar_obj.explode_array_attribute(spar_df3, 'placeSearchOpeningHours')

# placeSearchOpeningHours attribute struct columns extraction

struct_placeSearchOpeningHours_attr_list = ['date', 'opens', 'closes', 'isToday', 'isOpenForTheDay']
clp_df6 = clp_obj.extract_struct_attributes(clp_df5, 'placeSearchOpeningHours', struct_placeSearchOpeningHours_attr_list)
cogo_df6 = cogo_obj.extract_struct_attributes(cogo_df5, 'placeSearchOpeningHours', struct_placeSearchOpeningHours_attr_list)
okay_df6 = okay_obj.extract_struct_attributes(okay_df5, 'placeSearchOpeningHours', struct_placeSearchOpeningHours_attr_list)
spar_df5 = spar_obj.extract_struct_attributes(spar_df4, 'placeSearchOpeningHours', struct_placeSearchOpeningHours_attr_list)


# DATS > add all the missing columns with default values
            
dats_missing_columns_list = ['handoverServices', 'sellingPartners', 'placeSearchOpeningHours_date', \
                             'placeSearchOpeningHours_opens', 'placeSearchOpeningHours_opens',\
                             'placeSearchOpeningHours_closes', 'placeSearchOpeningHours_isToday',\
                             'placeSearchOpeningHours_isOpenForTheDay', 'placeSearchOpeningHours'\
                             ]
dats_df3 = dats_obj.add_default_string_column(dats_df2, dats_missing_columns_list)
logger.info(f'DATS post adding default columns')
logger.debug(dats_df3.printSchema())


# Organising schema
spar_df6 = spar_obj.organise_schema(spar_df5)
dats_df4 = dats_obj.organise_schema(dats_df3)


# Drop placeSearchOpeningHours 
drop_col_list = ('placeSearchOpeningHours', 'placeSearchOpeningHours_isOpenForTheDay', 'placeSearchOpeningHours_isToday')
clp_df7 = clp_obj.drop_columns(clp_df6, drop_col_list)
cogo_df7 = cogo_obj.drop_columns(cogo_df6, drop_col_list)
okay_df7 = okay_obj.drop_columns(okay_df6, drop_col_list)
spar_df7 = spar_obj.drop_columns(spar_df6, drop_col_list)
dats_df5 = dats_obj.drop_columns(dats_df4, drop_col_list)

logger.debug(f'Final CLP schema for clp_df7')
logger.debug(clp_df7.printSchema())
logger.debug(f'Final DATS schema for dats_df5')
logger.debug(dats_df5.printSchema())

    
# Union of all brands
merged_df = brands.union_brands(clp_df7, cogo_df7, okay_df7, spar_df7, dats_df5)
logger.info(f'merged_df count > {merged_df.count()}')

root
 |-- address: struct (nullable = true)
 |    |-- cityName: string (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- countryName: string (nullable = true)
 |    |-- houseNumber: string (nullable = true)
 |    |-- postalcode: string (nullable = true)
 |    |-- streetName: string (nullable = true)
 |-- branchId: string (nullable = true)
 |-- commercialName: string (nullable = true)
 |-- ensign: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- geoCoordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- isActive: boolean (nullable = true)
 |-- moreInfoUrl: string (nullable = true)
 |-- placeId: long (nullable = true)
 |-- placeSearchOpeningHours: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- closes: long (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- isOp

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


root
 |-- address: struct (nullable = true)
 |    |-- cityName: string (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- countryName: string (nullable = true)
 |    |-- houseNumber: string (nullable = true)
 |    |-- postalcode: string (nullable = true)
 |    |-- streetName: string (nullable = true)
 |-- branchId: string (nullable = true)
 |-- commercialName: string (nullable = true)
 |-- ensign: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- geoCoordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- handoverServices: string (nullable = true)
 |-- isActive: boolean (nullable = true)
 |-- moreInfoUrl: string (nullable = true)
 |-- placeId: long (nullable = true)
 |-- placeType: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- longName: string (nullable = true)
 |    |-- placeTypeDescription: string (nu

In [7]:
# Extract "postal_code" from address 
merged_df.printSchema()
merged_df.select('address.postalcode').show(10, 0)

root
 |-- address: struct (nullable = true)
 |    |-- cityName: string (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- countryName: string (nullable = true)
 |    |-- houseNumber: string (nullable = true)
 |    |-- postalcode: string (nullable = true)
 |    |-- streetName: string (nullable = true)
 |-- branchId: string (nullable = true)
 |-- commercialName: string (nullable = true)
 |-- ensign: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- geoCoordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- handoverServices: string (nullable = true)
 |-- isActive: boolean (nullable = true)
 |-- moreInfoUrl: string (nullable = true)
 |-- placeId: long (nullable = true)
 |-- placeType: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- longName: string (nullable = true)
 |    |-- placeTypeDescription: string (nu

In [8]:
# Create an object of class datasetTransform
dt_obj = datasetTransform(spark)
dt_obj.extract_postal_code(merged_df)

+----------+
|postalcode|
+----------+
|9300      |
|9300      |
|9300      |
|9300      |
|9300      |
|9300      |
|9300      |
|9300      |
|8830      |
|8830      |
+----------+
only showing top 10 rows



In [9]:
dt_obj.read_postal_code_mapping_config(postal_config_path)

+---------------+---------------------+
|province       |postalcode_range     |
+---------------+---------------------+
|Brussel        | 1000-1299           |
|Waals-Brabant  | 1300-1499           |
|Vlaams-Brabant | 1500-1999, 3000-3499|
|Antwerpen      | 2000-2999           |
|Limburg        | 3500-3999           |
|Luik           | 4000-4999           |
|Namen          | 5000-5999           |
|Henegouwen     | 6000-6599,7000-7999 |
|Luxemburg      | 6600-6999           |
|West-Vlaanderen| 8000-8999           |
|Oost-Vlaanderen| 9000-9999           |
+---------------+---------------------+



24/03/31 19:22:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 454091 ms exceeds timeout 120000 ms
24/03/31 19:22:26 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/31 19:22:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$